In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
import seaborn as sns 

In [5]:
# csv with adjusted MKTCAP
raw = pd.read_csv('MKTCAP_LN.csv', encoding = 'cp932').dropna()
raw.head()

,ddate,security_code,時価総額,売上,営業利益,純利益,総資産,現金･現金同等物,有価証券,流動資産,...,営業資産,営業負債,有形固定,使用資産,ROIC,EY,MAGIC,ROIC2,EY2,MKTCAP
0,2000,AAL,0.0000,14824.000,2368.000,1957.000,30616.000,1061.000,2344.000,8127.000,...,4722.000,-4027.000,11819.000,12514.000,0.189228,inf,inf,0.114236,0.401220,0.0000
1,2000,ABF,3385.8000,4406.000,327.000,138.000,3867.000,65.000,1133.000,2220.000,...,1022.000,-735.000,1459.000,1746.000,0.187285,0.096580,0.265137,0.111833,0.092430,3385.8000
2,2000,ACL,121.9050,245.360,13.484,8.232,146.453,8.461,0.000,86.766,...,78.305,-54.935,9.342,32.712,0.412203,0.110611,0.481594,0.390139,0.092395,121.9050
3,2000,ADB,0.0000,32.645,2.448,1.466,23.528,0.412,0.000,6.764,...,6.352,-5.785,16.087,16.654,0.146992,inf,inf,0.185581,0.528041,0.0000
4,2000,ADN,846.9825,115.721,34.225,24.126,274.148,45.815,3.054,91.628,...,42.759,-81.432,11.307,-27.366,-1.250639,0.040408,-1.085167,0.306294,0.041408,846.9825


In [30]:
# any zero or blow zero MKTCAP values
raw[raw['MKTCAP'] < 0][['security_code', '時価総額', 'MKTCAP']].head()

print ( raw.shape)
raw[raw['MKTCAP'] == 0][['security_code', '時価総額', 'MKTCAP']].shape

(13620, 76)


(3987, 3)

In [31]:
# csv with original MKTCAP
df = pd.read_csv('sqldataln.csv', encoding = 'cp932')
print (df.shape)
df.head()

(502814, 5)


,security_code,ddate,quarter,title,value
0,7DIG,2001,200101,ANNOUNCEMENT_DT,2.001070e+07
1,7DIG,2001,200101,BS_ACCT_NOTE_RCV,6.470000e-01
2,7DIG,2001,200101,BS_ACCT_PAYABLE,3.200000e-01
3,7DIG,2001,200101,BS_CASH_NEAR_CASH_ITEM,2.368000e+00
4,7DIG,2001,200101,BS_CUR_ASSET_REPORT,3.393000e+00


In [74]:
# aal = df[df['security_code'] == 'AAL']
# aal[aal['title'] == 'HISTORICAL_MARKET_CAP']

In [40]:
new = pd.merge(raw, df, how = 'left', left_on = ['security_code', 'ddate'], right_on = ['security_code', 'ddate'])

In [116]:
unadjust = new[new['title'] == 'HISTORICAL_MARKET_CAP'].dropna()
unadjust.head()

,ddate,security_code,時価総額,売上,営業利益,純利益,総資産,現金･現金同等物,有価証券,流動資産,...,使用資産,ROIC,EY,MAGIC,ROIC2,EY2,MKTCAP,quarter,title,value
21,2000,AAL,0.0000,14824.000,2368.000,1957.000,30616.000,1061.000,2344.000,8127.000,...,12514.000,0.189228,inf,inf,0.114236,0.401220,0.0000,200001,HISTORICAL_MARKET_CAP,22469.2680
51,2000,ABF,3385.8000,4406.000,327.000,138.000,3867.000,65.000,1133.000,2220.000,...,1746.000,0.187285,0.096580,0.265137,0.111833,0.092430,3385.8000,200001,HISTORICAL_MARKET_CAP,2652.1079
80,2000,ACL,121.9050,245.360,13.484,8.232,146.453,8.461,0.000,86.766,...,32.712,0.412203,0.110611,0.481594,0.390139,0.092395,121.9050,200001,HISTORICAL_MARKET_CAP,116.6681
109,2000,ADB,0.0000,32.645,2.448,1.466,23.528,0.412,0.000,6.764,...,16.654,0.146992,inf,inf,0.185581,0.528041,0.0000,200001,HISTORICAL_MARKET_CAP,20.2960
138,2000,ADN,846.9825,115.721,34.225,24.126,274.148,45.815,3.054,91.628,...,-27.366,-1.250639,0.040408,-1.085167,0.306294,0.041408,846.9825,200001,HISTORICAL_MARKET_CAP,884.2819


In [78]:
# any zero or below zero MKTCAP vlaue 
mktcap_value = unadjust[unadjust['title'] == 'HISTORICAL_MARKET_CAP'][['security_code', 'value']]
mktcap_value[mktcap_value['value'] == 0]
mktcap_value[mktcap_value['value'] < 0]

,security_code,value


In [133]:
collist = unadjust.columns.tolist()
raw2 = unadjust.drop(collist[-9: -1], axis = 1)

# x = data.iloc[:, :-1]
# y = data['value']

In [134]:
raw2.groupby('ddate').count()

,security_code,時価総額,売上,営業利益,純利益,総資産,現金･現金同等物,有価証券,流動資産,総長期投資,...,販管費,売上総利益率,営業利益率,株主還元,純現金資産,営業資産,営業負債,有形固定,使用資産,value
ddate,,,,,,,,,,,,,,,,,,,,,
2000,439,439,439,439,439,439,439,439,439,439,...,439,439,439,439,439,439,439,439,439,439
2001,467,467,467,467,467,467,467,467,467,467,...,467,467,467,467,467,467,467,467,467,467
2002,484,484,484,484,484,484,484,484,484,484,...,484,484,484,484,484,484,484,484,484,484
2003,486,486,486,486,486,486,486,486,486,486,...,486,486,486,486,486,486,486,486,486,486
2004,540,540,540,540,540,540,540,540,540,540,...,540,540,540,540,540,540,540,540,540,540
2005,620,620,620,620,620,620,620,620,620,620,...,620,620,620,620,620,620,620,620,620,620
2006,714,714,714,714,714,714,714,714,714,714,...,714,714,714,714,714,714,714,714,714,714
2007,778,778,778,778,778,778,778,778,778,778,...,778,778,778,778,778,778,778,778,778,778
2008,826,826,826,826,826,826,826,826,826,826,...,826,826,826,826,826,826,826,826,826,826


In [135]:
# remove all zero columns. i.e. keep columns that have more than one zero values 
data = raw2.loc[:, (raw2 != 0).any(axis = 0)]

# only keep the companeis that exist in both 2014 and 2015
x_2015= data[data['ddate'] == 2015].drop('value', axis = 1)
y_2016 = data[data['ddate'] == 2016][['security_code','value']]

data2 = pd.merge(x_2015, y_2016, how = 'inner', on = ['security_code'])
print (data2.shape)

(1060, 62)
